In [7]:
import numpy as np
import torch as ch
from tqdm import tqdm
from pathlib import Path
from typing import List, Optional
import torch

from unlearning.unlearning_algos.utils import get_margin
dataset_name = "CIFAR10"

ULIRA_BASE_DIR = Path(
    "/n/home04/rrinberg/data_dir__holylabs/unlearning/precomputed_models/ULIRA_clean/"
)

masks_path = ULIRA_BASE_DIR  / "training_masks.npy"
masks = np.load(masks_path)

#
from unlearning.auditors.utils import loader_factory
ds_name_ = "CIFAR10"
train_loader = loader_factory(ds_name_, indexed=True)
val_loader = loader_factory(ds_name_, split="val", indexed=True)
eval_set_inds = np.arange(len(train_loader.dataset) + len(val_loader.dataset))
eval_loader = loader_factory(ds_name_,
                            split="train_and_val",
                            indices=eval_set_inds,
                            indexed=True)

train_targets = train_loader.dataset.original_dataset.targets
val_targets = val_loader.dataset.original_dataset.targets



No augmentation
Files already downloaded and verified


/n/holylabs/LABS/vadhan_lab/Lab/rrinberg/envs/unlearning_3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


No augmentation
Files already downloaded and verified
Making train-and-val dataset
No augmentation
Files already downloaded and verified
No augmentation
Files already downloaded and verified


In [8]:
def get_train_logits(model_index, dataset_name):
    train_logits_path = ULIRA_BASE_DIR / dataset_name / f"train_logits_{model_index}.pt"
    if not train_logits_path.exists():
        return None
    return torch.load(train_logits_path)

def get_val_logits(model_index, dataset_name):
    val_logits_path = ULIRA_BASE_DIR / dataset_name / f"val_logits_{model_index}.pt"
    return torch.load(val_logits_path)


def get_logits(model_index, dataset_name):
    train_logits_path = ULIRA_BASE_DIR / dataset_name / f"train_logits_{model_index}.pt"
    val_logits_path = ULIRA_BASE_DIR / dataset_name / f"val_logits_{model_index}.pt"
    if not train_logits_path.exists():
        return None
    return torch.load(train_logits_path), torch.load(val_logits_path)


def compute_margins(logit, true_label):
    logit_other = logit.clone()
    logit_other[true_label] = -np.inf

    return logit[true_label] - logit_other.logsumexp(dim=-1)


def get_margins(model_ind):
    # HACK
    fake_ds_name = "final_models"
    # HACK 
    train_logits = get_train_logits(model_ind, fake_ds_name)
    val_logits = get_val_logits(model_ind, fake_ds_name)
    train_margins = [
        compute_margins(logit, target)
        for (logit, target) in zip(train_logits, train_targets)
    ]
    val_margins = [
        compute_margins(logit, target)
        for (logit, target) in zip(val_logits, val_targets)
    ]
    return np.array(train_margins), np.array(val_margins)


In [9]:
all_models = ULIRA_BASE_DIR / "final_models"
all_models = all_models.glob("sd_*.pt")
nums = []
for model in all_models:

    #print(model)
    #print(model.name.split("_"))
    num = int(str(model.name).split("_")[1])
    nums.append(num)
    #print(num)
#nums = [int(str(model).split("_")[1]) for model in all_models]
nums = sorted(nums)
expected = set(range(len(nums)))
missing = expected - set(nums)
print(f"Missing: {sorted(missing)}")

Missing: []


In [11]:
model_count = masks.shape[0]
for model_ind in range(0, 525):

    train_path = ULIRA_BASE_DIR / "final_models" / f"train_margins_{model_ind}.pt"
    val_path = ULIRA_BASE_DIR / "final_models" / f"val_margins_{model_ind}.pt"
    if Path(train_path).exists():
        print(f"already computed {model_ind}")
        continue

    if model_ind % 10 == 0:
        print(model_ind)
    try:
        train_margins, val_margins = get_margins(model_ind)
    except Exception as e:
        print(e)
        break
        print(f"fail on {model_ind}")
        continue
    # torch save

    torch.save(train_margins, train_path)
    torch.save(val_margins, val_path)
# TODO
# 1: save all the margins
# 2.
#for model_ind in


already computed 0
already computed 1
already computed 2
already computed 3
already computed 4
already computed 5
already computed 6
already computed 7
already computed 8
already computed 9
already computed 10
already computed 11
already computed 12
already computed 13
already computed 14
already computed 15
already computed 16
already computed 17
already computed 18
already computed 19
already computed 20
already computed 21
already computed 22
already computed 23
already computed 24
already computed 25
already computed 26
already computed 27
already computed 28
already computed 29
already computed 30
already computed 31
already computed 32
already computed 33
already computed 34
already computed 35
already computed 36
already computed 37
already computed 38
already computed 39
already computed 40
already computed 41
already computed 42
already computed 43
already computed 44
already computed 45
already computed 46
already computed 47
already computed 48
already computed 49
already co

In [12]:
# ORACLE MODEL margin computation
dataset_name = "CIFAR10"
train_f = ULIRA_BASE_DIR / "final_models"# / "train_margins_all.pt"
val_f = ULIRA_BASE_DIR / "final_models" #/ "val_margins_all.pt"

def sort_key(x):
    return int(x.name.split("_")[-1][:-len(".pt")])
set_name = "train"


for set_name in ["train", "val"]:
    #if True:
    print(set_name)
    #margin_fs = list(train_f.glob(f"*_ulira_margins*"))
    margin_fs = list(train_f.glob(f"{set_name}_margins_*.pt"))
    # drop "train_margin_all.pt"
    margin_fs = [f for f in margin_fs if "all" not in f.name]
    margin_fs = sorted(margin_fs, key=sort_key)

    #print(margin_fs[-1].name)
    
    loaded = []
    for i, f in enumerate(margin_fs):
        margin_group = ch.load(f) 
        #print(f"margin_group: {margin_group.shape}") # 50 x 60k
        
        loaded.append(ch.load(f))
    # stack the margins
    #all_margins = ch.cat(loaded, dim=0)
    all_margins = np.array(loaded)#ch.cat(loaded, dim=0)
    print(all_margins.shape)

    # save the margins
    ch.save(ch.tensor(all_margins), train_f / f"{set_name}_margins_all.pt")

train
(525, 50000)
val
(525, 10000)
